In [1]:
import tensorflow as tf
import numpy as np
import gym
import gym.spaces
from pylab import *
import scipy.misc

In [16]:
%matplotlib notebook

In [17]:
G = 0.9
N_ACTIONS = 6
ACTIONS = np.arange(N_ACTIONS)

## Network setup

In [854]:
tf.reset_default_graph()
sess = tf.Session()

graph_r = tf.placeholder(tf.float32, [None])
graph_lr = tf.Variable(1e-4)
graph_s = tf.placeholder(tf.float32, [None, 84, 84, 4])
graph_action = tf.placeholder(tf.int64, [None])
graph_advantage = tf.placeholder(tf.float32, [None])

x = tf.layers.conv2d(
        inputs=graph_s,
        filters=32,
        kernel_size=8,
        strides=4,
        activation=tf.nn.relu)

x = tf.layers.conv2d(
        inputs=x,
        filters=64,
        kernel_size=4,
        strides=2,
        activation=tf.nn.relu)

x = tf.layers.conv2d(
        inputs=x,
        filters=64,
        kernel_size=3,
        strides=1,
        activation=tf.nn.relu)

w, h, f = x.shape[1:]
x = tf.reshape(x, [-1, int(w * h * f)])

x = tf.layers.dense(
        inputs=x,
        units=512,
        activation=tf.nn.relu)

a_logits = tf.layers.dense(
        inputs=x,
        units=N_ACTIONS,
        activation=None)

a_softmax = tf.nn.softmax(a_logits)

graph_v = tf.layers.dense(
    inputs=x,
    units=1,
    activation=None)
graph_v = graph_v[:, 0]

p = 0
for i in range(N_ACTIONS):
    p += tf.cast(tf.equal(graph_action, i), tf.float32) * a_softmax[:, i]
nlp = -1 * tf.log(p)
loss = tf.reduce_mean(nlp * graph_advantage)
t = tf.train.AdamOptimizer(learning_rate=graph_lr).minimize(loss)

# TODO: reduce_mean or reduce_sum?
loss2 = tf.reduce_mean((graph_r - graph_v) ** 2)
t2 = tf.train.RMSPropOptimizer(learning_rate=graph_lr).minimize(loss2)

## Action probability selection test

In [461]:
sess.run(tf.global_variables_initializer())
s = np.random.random((1, 84, 84, 4))
s = np.vstack([s] * N_ACTIONS)
expected = -1 * np.log(sess.run(a_softmax, feed_dict={graph_s: s})[0])
actual = sess.run(nlp, feed_dict={graph_s: s, graph_action: np.arange(N_ACTIONS)})
np.testing.assert_allclose(expected, actual)

## Basic policy network training test

In [187]:
sess.run(tf.global_variables_initializer())
n = 5
s = np.random.random((1, 84, 84, 4))
s = np.vstack([s] * n)

# encourage action 0
print(sess.run(a_softmax, feed_dict={graph_s: s})[0])
a = [0] * n
ad = [1] * n
for i in range(3):
    sess.run(t, feed_dict={graph_s: s, graph_action: a, graph_advantage: ad})
    print(sess.run(a_softmax, feed_dict={graph_s: s})[0])
    
# now the probability of action[0] will be very close to 1,
# so the gradients will be too small to work
# so reset
sess.run(tf.global_variables_initializer())
print()

# discourage action 0
print(sess.run(a_softmax, feed_dict={graph_s: s})[0])
a = [0] * n
ad = [-1] * n
for i in range(3):
    sess.run(t, feed_dict={graph_s: s, graph_action: a, graph_advantage: ad})
    print(sess.run(a_softmax, feed_dict={graph_s: s})[0])

[ 0.17372544  0.18403009  0.16954473  0.15695326  0.14080301  0.17494345]
[ 0.69514018  0.05938208  0.05715563  0.06667018  0.06778827  0.05386371]
[  9.99270022e-01   1.21817924e-04   8.89854637e-05   2.19529844e-04
   2.00164999e-04   9.94842703e-05]
[  1.00000000e+00   2.65501710e-09   1.43055701e-09   1.13996155e-08
   9.30531918e-09   2.65303290e-09]

[ 0.16540162  0.15732603  0.17260852  0.14700982  0.1905255   0.16712849]
[ 0.13441537  0.16121353  0.17941232  0.15326598  0.19829994  0.17339279]
[ 0.10914944  0.16310006  0.18478177  0.15826537  0.20659208  0.17811126]
[ 0.08741591  0.16315605  0.1900837   0.16082349  0.2164374   0.18208353]


In [638]:
sess.run(tf.global_variables_initializer())
sess.run(tf.assign(graph_lr, 1e-3))
print(sess.run(a_softmax, feed_dict={graph_s: s})[0])

[ 0.14056821  0.17801271  0.16285437  0.18665454  0.15588795  0.17602223]


In [748]:
sess.run(t1, feed_dict={graph_s: s, graph_action: [0], graph_advantage: [-1]})
print(sess.run(a_softmax, feed_dict={graph_s: s})[0])

[  9.95181739e-01   8.53346195e-04   4.23620862e-04   1.45785592e-03
   7.20662822e-04   1.36271701e-03]


## Basic value network training test

In [539]:
s = np.random.normal(size=(5, 84, 84, 4))
r = np.array([1, 2, 3, 4, 5]) / 5.0
print(r)
sess.run(tf.global_variables_initializer())
sess.run(graph_v, feed_dict={graph_s: s})

[ 0.2  0.4  0.6  0.8  1. ]


array([-0.25281048, -0.16718042, -0.33672947, -0.19710909, -0.30827755], dtype=float32)

In [608]:
sess.run(t2, feed_dict={graph_s: s, graph_r: r})
sess.run([loss2, graph_v], feed_dict={graph_s: s, graph_r: r})

[5.6838653e-05,
 array([ 0.1950734 ,  0.39835683,  0.6061551 ,  0.80783087,  1.01257038], dtype=float32)]

## Utils

In [24]:
def copy_network():
    n1_dict = {var.name: var for var in tf.trainable_variables() if 'network1' in var.name}
    n2_dict = {var.name: var for var in tf.trainable_variables() if 'network2' in var.name}
    copy_ops = []
    for name, var in n2_dict.items():
        op = var.assign(n1_dict[name.replace('network2', 'network1')].value())
        copy_ops.append(op)
    sess.run(copy_ops)
    
def with_prob(p):
    if np.random.random() < p:
        return True
    else:
        return False

def discount_rewards(r, G):
    r2 = np.zeros_like(np.array(r).astype(np.float32))
    r2[-1] = r[-1]
    for i in range(len(r2)-2, -1, -1):
        r2[i] = G * r2[i+1]
    return r2

def rewards_to_returns(r, G):
    r2 = np.zeros_like(np.array(r).astype(np.float32))
    r2[-1] = r[-1]
    for i in range(len(r2)-2, -1, -1):
        r2[i] = r[i] + G * r2[i+1]
    return r2

In [765]:
print(discount_rewards([0, 0, 0, 1], 0.9))
print(rewards_to_returns([1, 2, 3, 4], 0.9))

[ 0.72899997  0.81        0.89999998  1.        ]
[ 8.14599991  7.94000006  6.5999999   4.        ]


In [846]:
def get_o(a):
    os = []
    rs = []
    for i in range(4):
        o1, r1, _, _ = env.step(a)
        env.render()
        o2, r2, done, _ = env.step(a)
        env.render()
        o = np.maximum(o1, o2)
        o = np.mean(o, axis=2)
        o = scipy.misc.imresize(o, (84, 84))
        os.append(o)
        rs.extend([r1, r2])
    os = np.stack(os, axis=-1)
    # TODO: is this necessary even with batchnorm?
    os = os / 255
    # TODO: is summing the right thing to do?
    r = np.sum(rs)
    return os, r, done

## Dummy environment

In [741]:
class DummyEnv:
    action_space = gym.spaces.discrete.Discrete(2)
    N_STATES = 3
    N_REWARDED_SEQUENCES = int(N_ACTIONS ** N_STATES / 10)
    
    def __init__(self):
        self.rewarded_sequences = np.clip(np.random.normal(loc=int(N_ACTIONS / 2), size=(self.N_REWARDED_SEQUENCES, self.N_STATES)).astype(np.int), 0, N_ACTIONS)
        self.rewarded_sequences = np.unique(self.rewarded_sequences, axis=0)
        self.o = {}
        # look at all unique sequence prefixes
        for i in range(1, self.N_STATES + 1):
            prefixes = np.unique(env.rewarded_sequences[:, :i], axis=0)
            for p in prefixes:
                self.o[tuple(p)] = np.random.normal(size=(84, 84, 4))
        self.default_o = np.random.normal(size=(84, 84, 4))
        self.init_o = np.random.normal(size=(84, 84, 4))
    
    def reset(self):
        self.i = 0
        self.history = []
        self.possible_sequences = np.copy(self.rewarded_sequences)
        return self.init_o
    
    def step(self, a):
        self.history.append(a)
        
        ps_new = []
        l = len(self.history)
        for seq in self.possible_sequences:
            if np.array_equal(seq[:l], self.history):
                ps_new.append(seq)
        self.possible_sequences = ps_new
        if len(self.possible_sequences) > 0:
            reward = 1
        else:
            reward = 0
            
        if tuple(self.history) in self.o:
            o = self.o[tuple(self.history)]
        else:
            o = self.default_o

        if self.i != self.N_STATES - 1:
            done = False
            self.i += 1
        else:
            done = True

        return o, reward, done, None

In [742]:
env = DummyEnv()

In [743]:
for i in range(N_ACTIONS):
    env.reset()
    env.step(i)
    print(len(env.possible_sequences))

0
2
5
11
11
7
3
0
0


In [744]:
# Initial state same?
o = env.reset()
print(np.sum(o))
o = env.reset()
print(np.sum(o))

-225.519265304
-225.519265304


In [745]:
# Default state the same?
env.reset()
o, _, _, _ = env.step(0)
print(np.sum(o))
env.reset()
o, _, _, _ = env.step(8)
print(np.sum(o))

37.8046131907
37.8046131907


In [747]:
# Always the same state returned for a given sequence?
env.reset()
o, _, _, _ = env.step(5)
print(np.sum(o))
env.reset()
o, _, _, _ = env.step(5)
print(np.sum(o))

env.reset()
env.step(5)
o, _, _, _ = env.step(5)
print(np.sum(o))
env.reset()
env.step(5)
o, _, _, _ = env.step(5)
print(np.sum(o))

-136.684018484
-136.684018484
-223.833923894
-223.833923894


In [749]:
def test_seq(seq):
    for a in seq:
        print("Took action", a)
        _, reward, done, _ = env.step(a)
        print("Got reward ", reward)
        print("Done:", done)

env.reset()
test_seq([0, 0, 0])
print()
env.reset()
test_seq(env.rewarded_sequences[0])

Took action 0
Got reward  0
Done: False
Took action 0
Got reward  0
Done: False
Took action 0
Got reward  0
Done: True

Took action 1
Got reward  1
Done: False
Took action 3
Got reward  1
Done: False
Took action 3
Got reward  1
Done: True


## Testing proper

In [818]:
env = DummyEnv()

In [841]:
env = gym.make('Pong-v0')

[2017-07-15 18:28:31,730] Making new env: Pong-v0


### Manual testing

In [ ]:
sess.run(tf.global_variables_initializer())
sess.run(t, feed_dict={graph_s: [env.o[2]], graph_action: [1], graph_advantage: [1]})
sess.run(a_softmax, feed_dict={graph_s: env.o})

### Testing with environment

In [850]:
rpe = [0]
sess.run(tf.global_variables_initializer())

In [852]:
for i in range(1):
    env.reset()
    o, _, _ = get_o(0) # do nothing
    done = False
    rewards = []
    actions = []
    states = []
    while not done:
        states.append(o)
        a_p = sess.run(a_softmax, feed_dict={graph_s: [o]})[0]
        a = np.random.choice(ACTIONS, p=a_p)
        actions.append(a)
        
        o, r, done = get_o(a)
        rewards.append(r)
        
        if r != 0:
            rpe.append(rpe[-1] + 0.1 * (np.sum(rewards) - rpe[-1]))
            returns = rewards_to_returns(rewards, G=0.9)
            sess.run([t, t2], feed_dict={graph_s: states, graph_action: actions, graph_advantage: returns, graph_r: returns})
            rewards = []
            actions = []
            states = []

<IPython.core.display.Javascript object>


In [853]:
plt.plot(rpe)

## Old code

In [131]:
t = 0
t_max = 100

s = np.zeros(10)
r1 = np.zeros(10)
a = np.zeros(10)
r2_list = []

for i in range(10):
    t_start = t
    s[t] = env.reset()
    done = False
    
    while True:
        a_p = sess.run(ap_op, feed_dict={s_op: s})
        a[t] = np.random.choice(ACTIONS, p=a_p)
        s[t+1], r[t], done, _ = env.step(a[t])
        t += 1        
    if done or t - t_start == t_max:
        break
    
    if done:
        r2 = 0
    else:
        r2 = sess.run(v_op, feed_dict={s_op: s[t]})
    
    for i in range(t-1, t_start-1, -1):
        r2_list.append(r[i] + g * r2)
    
    
    sess.run(t1, feed_dict={s_op: s, big_r_op: r})
    sess.run(t2, feed_dict={s_op: s, big_r_op: r})

0
0
0
0
0
0
0
0
0
0


In [110]:
observation.shape
action = env.action_space.sample()
os = []
for i in range(4):
    o1, _, _, _ = env.step(action)
    o2, _, _, _ = env.step(action)
    o = np.maximum(o1, o2)
    o = np.mean(o, axis=2)
    o = scipy.misc.imresize(o, (84, 84))
    os.append(o)
os = np.stack(os, axis=-1)

In [80]:
os = np.resize(os, (1, 84, 84, 4))
print(os.shape)
print(sess.run(q['network1'], feed_dict={s: os}))
print(sess.run(qi['network1'], feed_dict={s: os}))

(1, 84, 84, 4)
[[  0.69894785  -0.22462213   5.59576368  -7.67884588  16.15483665
    2.82746601   0.95167667   5.31193829   0.30126441]]
[ 16.15483665]
[4]
